In [1]:
from PIL import Image

In [22]:
font = list(range(0x7f))
for a in range(0x20, 0x7f):
    font[a] = Image.open("font/" + hex(a)[2:] + ".png")

In [25]:
def gen_image(text):
    width = 0
    for a in text:
        width += font[ord(a)].size[0] + 1
    height = font[ord('A')].size[1]
    img = Image.new(mode = "RGBA", size = (width, height), color = (255, 255, 255, 100))
    
    left = 0
    for a in text:
        letter = font[ord(a)]
        img.paste(letter, (left, img.size[1] - height))
        left += letter.size[0] + 1
    return img

In [27]:
gen_image('ABCDEFG& ?')